In [27]:
import copy
import random as rd
import csv

In [28]:
#Group1
path1 = {'node1':90,'node2':30,'node3':60,'node4':60,'node5':40,'node6':20,'node7':60,'node8':20,'end':999}
path2 = {'node9':90,'node10':30,'node11':60,'node12':60,'node13':40,'node14':20,'node15':60,'node16':20,'end':999}
path3 = {'node17':90,'node18':30,'node19':60,'node20':60,'node21':40,'node22':20,'node23':60,'node24':20,'end':999}

#Group2
path4 = {'node25':90,'node26':30,'node27':60,'node28':60,'node29':40,'node30':20,'node31':60,'node32':20,'end':999}
path5 = {'node33':90,'node34':30,'node35':60,'node36':60,'node37':40,'node38':20,'node39':60,'node40':20,'end':999}
path6 = {'node41':90,'node42':30,'node43':60,'node44':60,'node45':40,'node46':20,'node47':60,'node48':20,'end':999}

#Group3
path7 = {'node49':90,'node50':30,'node51':60,'node52':60,'node53':40,'node54':20,'node55':60,'end':999}
path8 = {'node56':90,'node57':30,'node58':60,'node59':60,'node60':40,'node61':20,'node62':60,'end':999}
path9 = {'node63':90,'node64':30,'node65':60,'node66':60,'node67':40,'node68':20,'node69':60,'end':999}

#Fast Track
path10 = {'node49':90,'node27':60,'node3':60,'node57':30,'node64':30,'node36':60,'node12':60,'node51':60,'node37':40,'node13':40,
          'node66':60,'node46':20,'node22':20,'node53':40,'node61':20,'node69':60,'end':999}

In [29]:
nodePaths = {'path1':path1,
               'path2':path2,
               'path3':path3,
               'path4':path4,
               'path5':path5,
               'path6':path6,
               'path7':path7,
               'path8':path8,
               'path9':path9,
               'path10':path10
              } #nodename : service tieme in seconds
nodeDwellRange = 10 #seconds eg. 40 second dwell time with 10 second range, people can stay between 30-50 seconds

liftRoundTime = 45 #seconds
liftCapacity = 12 #people

simulationTime = 1800 #seconds



In [50]:
class person:
    def __init__(self,personIndex,timeAtNode,pathList,nodeDetail):
        self.idx = personIndex
        self.curNode = pathList[0]
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.nodeList = pathList
        self.node_detail = nodeDetail 
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        lowRange = self.node_detail[self.curNode]-nodeDwellRange
        highRange = self.node_detail[self.curNode] + nodeDwellRange
        timeToSpendAtCurNode = rd.randrange(lowRange,highRange+1,1)
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == self.nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return
            self.curNodeIdx+=1
            self.curNode = self.nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            return

In [54]:
#simulate arrival time of people coming in a lift and moving through from one to the next and calculate occupancy of each node over time

#keep track of time when people arrive at a node, where they are and when they should leave.
numberOfPeople = 0
peopleList = []
nodeOccupancies = {}
for curTime in range(simulationTime):

    if curTime not in nodeOccupancies:
        nodeOccupancies[curTime] = {}
        for key in nodePaths:
            nodeOccupancies[curTime]['end'] = 0
            for paths in list(nodePaths[key])[:-1]:
                nodeOccupancies[curTime][paths] = 0
    

    for idx,paths in enumerate(nodePaths):
        offset = idx * 10
        nodeDetails = nodePaths[paths]
        nodeList = list(nodeDetails)[:-1]

        if (curTime+offset)%liftRoundTime == 0:# People Arrive at node 1
            for personIndex in range(liftCapacity):
                peopleList.append(person(len(peopleList),curTime,nodeList,nodeDetails))

    for agent in peopleList:
        agent.updateAgentLocation(curTime)
        agentLocation = agent.curNode
        nodeOccupancies[curTime][agentLocation] += 1



In [56]:
# Get headers from one of the inner dictionaries
fieldnames = ['time'] + list(next(iter(nodeOccupancies.values())).keys())

with open('nodeStaticCalcsOutput_Multi.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for time, node_values in nodeOccupancies.items():
        row = {'time': time}
        row.update(node_values)
        writer.writerow(row)